First we need to load the dataset and needed libraries, along with setting only the records responding to the small version available

In [ ]:
import pandas as pd 
import numpy as np

df = pd.read_csv('../../data/filtered_tracks.csv')
print(df.head())

In [ ]:
def count_classes(array):
    dict_clasess={}

    for row in  array:
        str_row=str(row)
        if str_row in dict_clasess:
            dict_clasess[str_row]=dict_clasess[str_row]+1
        else:
            dict_clasess[str_row]=1
    return dict_clasess

def create_dataset(metadane="../../data/",input_dataset="../../data/tracks.csv",audio_folder="../../data/fma_small",n_classes=10,num_ele_per_class=1500):
    genres=metadane+"genres.csv"
    genres=pd.read_csv(genres)
    all_tracks=pd.read_csv(input_dataset)
    labels=all_tracks.values
    label_genre_top=np.where(labels[0,:]=="genre_top")
    label_genres=np.where(labels[0,:]=="genres")
    label_genres_all=np.where(labels[0,:]=="genres_all")
    label_subset=np.where(labels[0,:]=="subset")
    label_id=np.where(labels[0,:]=="id")
    label_bit_rate=np.where(labels[0,:]=="bit_rate")

    all_id=labels[2:, 0]
    all_label_subset=labels[2:,label_subset[0][0]]
    all_genre_top=labels[2:,label_genre_top[0][0]]
    all_genres=labels[2:,label_genres[0][0]]
    all_genres_all=labels[2:,label_genres_all[0][0]]
    number=6
    path_array=[]
    folder_path=audio_folder
    for id in all_id:
        id_len=len(str(id))
        formatted_id=str(id)
        if id_len!=number:
            formatted_id=str(id).zfill(number)
        st=formatted_id[0:3:1]
        sub_folder_path="\\"+st+"\\"+formatted_id+".mp3"
        mp3_path=folder_path+sub_folder_path
        path_array.append(mp3_path)
    songs=pd.DataFrame({
    "genre_top":all_genre_top,
    "genres":all_genres,
    "genres_all":all_genres_all,
    "subset":all_label_subset,
    "id":all_id,
    'path':path_array
    })
    return songs

df = create_dataset()
print(df.size)
print(df.head())

In [ ]:
genre_counts = df['genre_top'].value_counts()
sufficient_genres = genre_counts[genre_counts >= 1200].index

balanced_df = df[df['genres'].isin(sufficient_genres)].groupby('genre_top')
print(balanced_df.head())

# Preparation

To prepare the .csv file for the script responsible for creating the spectrograms you sould run this code:

In [ ]:
df['genres'] = df['genres'].replace('Old-Time / Historic', 'Old-Time')

df['audioPath'] = df['audioPath'].where(df['subset'].ne('small'), df['audioPath'].replace('fma_medium', 'fma_small', regex=True))

df['audioPath'] = '..\\..\\data\\' + df['audioPath']
df['spectrogramPath'] = '..\\..\\data\\spectrograms\\' + df['genres'] + '\\' + df['id'].astype('str') + '.png'
df = df[df.genres != 'Easy Listening']
genreList = '..\\..\\data\\spectrograms\\' + df['genres'].unique()

df.to_csv('..\\..\\data\\tracks_cleaned_upg.csv', index=False)

# Spectrogram .csv

This script will edit the previously created one, to allow us to process the spectrograms

In [ ]:
import os
df = pd.read_csv('..\\..\\data\\tracks_cleaned_upg.csv')
df = df[df.subset == 'small']
df = df.drop(['num_genres', 'subset', 'listens', 'favorites', 'audioPath'], axis=1)
df = df.drop(df[~df['spectrogramPath'].apply(os.path.exists)].index)
print(df)
df.reset_index()
# print(df)
df.to_csv('..\\..\\data\\spectrograms.csv', index=False)